# Formação Cientista de Dados - DSA

### Big Data Real-Time Analytics com Python e Spark

## Projeto com Feedback 4 - Prevendo Customer Churn em Operadoras de Telecom

### Leonardo Molero

# Análise Exploratória

In [ ]:
# Importação pacotes iniciais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [ ]:
# Faz ajustes para não exibir warnings
warnings.filterwarnings("ignore")

# Parametriza impressão dos gráficos dentro do notebook
%matplotlib inline

In [ ]:
# Carrega o dados de treino colocando a primeira coluna como index
df = pd.read_csv('dados/projeto4_telecom_treino.csv',index_col=[0])

In [ ]:
# Checa o tamanho do dataframe
print(df.shape)

In [ ]:
# Visualiza os dados treino
df.head(10)

In [ ]:
# Verifica os tipos das colunas
df.dtypes

In [ ]:
# Verifica estatísticas das colunas numéricas
df.describe()

In [ ]:
# Distribuição da variável alvo
df.groupby('churn').size()

In [ ]:
# Plota a distribuição da variável alvo
sns.countplot(x='churn',data=df,palette="Blues")

In [ ]:
# Verifica variáveis tipo texto 
print(df.groupby('area_code').size())
print(df.groupby('international_plan').size())
print(df.groupby('voice_mail_plan').size())

In [ ]:
# Plota um gráfico de relação de todas as variáveis do dataset
sns.pairplot(df, hue='churn')

In [ ]:
# Verifica distribuição das variáveis numéricas
df.hist(figsize=(12, 8),layout=(3,5))
plt.show()

In [ ]:
# Verifica os gastos de ligação / horário por estado

f, ax = plt.subplots(figsize=(7, 15))

sns.set_color_codes("pastel")
sns.barplot(x="total_day_charge", y="state", data=df,
            label="Dia", color="b")


sns.set_color_codes("muted")
sns.barplot(x="total_eve_charge", y="state", data=df,
            label="Tarde", color="b")

sns.set_color_codes("dark")
sns.barplot(x="total_night_charge", y="state", data=df,
            label="Noite", color="b")

sns.set_color_codes("colorblind")
sns.barplot(x="total_intl_charge", y="state", data=df,
            label="Interurbano", color="b")

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
ax.set(xlabel="Gastos Ligação por Horário")
sns.despine(left=True, bottom=True)

In [ ]:
# # Ligações para atendimento ao consumidor por estado
f, ax = plt.subplots(figsize=(16, 8))
sns.lineplot(x='state',y='number_customer_service_calls',data=df)

In [ ]:
# Ligações para atendimento ao consumidor X rotatividade
sns.boxplot(y='number_customer_service_calls',x='churn',data=df)

In [ ]:
# Verifica a correlação das variáveis numéricas
fig,ax = plt.subplots(figsize=(12,8))
sns.heatmap(df.corr(),annot=True,cmap='viridis')

In [ ]:
# Procura por valores nulos
df.isnull().values.any()

# Tratamento dos Dados

In [ ]:
# Retira as varíaveis com alta correlação
df = df.drop(['total_day_minutes','total_eve_minutes','total_night_minutes','total_intl_minutes'],axis=1)

In [ ]:
# Verifica novamente a correlação das variáveis
fig,ax = plt.subplots(figsize=(12,8))
sns.heatmap(df.corr(),annot=True,cmap='viridis')

In [ ]:
# Converte a variável alvo para númerica (0 e 1)

# Cria função para substituir varíaveis "Sim" e "Não" por 1 e 0
def binar(x):
    if x == 'yes':
        return 1
    else:
        return 0

# Aplica função na coluna alvo
df['churn'] = df['churn'].apply(binar)

In [ ]:
# Converte demais variáveis de sim e não para numérica (1 e 0)
df['international_plan'] = df['international_plan'].apply(binar)
df['voice_mail_plan'] = df['voice_mail_plan'].apply(binar)

In [ ]:
# Verifica a importância das variáveis com o RandomFlorest

from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()

var_n = df.drop(['churn','state','area_code'],axis=1)
target = df['churn']

rfc.fit(var_n,target)

sns.barplot(x=rfc.feature_importances_, y=var_n.columns,palette="Blues_d")

# Construção do Modelo

In [ ]:
# Cria os datasets de variáveis / alvo
# Serão descartadas as varíaveis não numéricas "state" e "area_code"
# Serão descartadas as varíaveis numéricas "voice_mail_plan" e "number_vmail_messages" (baixa importância)

X = df.drop(['churn','state','area_code','voice_mail_plan','number_vmail_messages'],axis=1)
y = df['churn']

In [ ]:
# Treina Modelo Regressão Logística

# Importação dos módulos
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# Definindo os valores para o número de folds
num_folds = 20

# Separando os dados em folds
kfold = KFold(num_folds, True)

# Criando o modelo
modelo = LogisticRegression()

# Cross Validation
resultado = cross_val_score(modelo, X, y, cv = kfold)

# Print do resultado
print("Acurácia Modelo 1: %.3f%%" % (resultado.mean() * 100))

In [ ]:
# Treina o modelo
modelo.fit(X,y)

# Testa o modelo

In [ ]:
# Importa dados de teste

teste = pd.read_csv('dados/projeto4_telecom_teste.csv',index_col=[0])

In [ ]:
# Prepara dados de teste

teste = teste.drop(['total_day_minutes','total_eve_minutes','total_night_minutes','total_intl_minutes'],axis=1)

teste['churn'] = teste['churn'].apply(binar)
teste['international_plan'] = teste['international_plan'].apply(binar)

In [ ]:
# Realiza as previsões

var = teste.drop(['churn','state','area_code','voice_mail_plan','number_vmail_messages'],axis=1)
alvo = teste['churn']

teste_prev = modelo.predict(var)

In [ ]:
# Verifica a performance do modelo nos dados de teste

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

report = classification_report(alvo, teste_prev)
matrix = confusion_matrix(alvo, teste_prev)

print(report)
print('\n')
print(matrix)